In [1]:
from mobilede_utils import request_car_detail_page

html = request_car_detail_page('358672814')

with open('car.html', 'w') as f:
    f.write(html)

In [2]:
from car_mapping import CAR_MAKE_MAP, CAR_MAKE_MODEL_MAP
from mobilede_utils import request_search_page

search_page = request_search_page('volkswage', 'id.3')

with open('search.html', 'w') as f:
    f.write(search_page)

TypeError: request_mobile_de_page() takes 1 positional argument but 2 were given

In [3]:
from mobilede_utils import request_mobile_de_page

url = '/fahrzeuge/details.html?id=372947832&cn=DE&fuels=ELECTRICITY&isSearchRequest=true&pageNumber=1&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&action=topOfPage&top=1:1&searchId=6421ed5b-1df1-1ccc-bd0c-b44042c43503&ref=srp'
html = request_mobile_de_page(url)

In [99]:
from mobilede_parser import extract_car_data

extract_car_data(html)

ImportError: cannot import name 'extract_car_data2' from 'mobilede_parser' (/home/toniramo/Documents/intro_to_DS_2023p1/project/importEVorNot/mobile_de/mobilede_parser.py)

In [79]:
%history

from mobilede_utils import request_mobile_de_page
from mobilede_parser import extract_car_data

url = 'https://suchen.mobile.de/fahrzeuge/details.html?id=372947832&cn=DE&fuels=ELECTRICITY&isSearchRequest=true&pageNumber=1&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&action=topOfPage&top=1:1&searchId=6421ed5b-1df1-1ccc-bd0c-b44042c43503&ref=srp'
html = request_mobile_de_page(url)
from mobilede_utils import request_mobile_de_page
from mobilede_parser import extract_car_data

url = 'fahrzeuge/details.html?id=372947832&cn=DE&fuels=ELECTRICITY&isSearchRequest=true&pageNumber=1&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&action=topOfPage&top=1:1&searchId=6421ed5b-1df1-1ccc-bd0c-b44042c43503&ref=srp'
html = request_mobile_de_page(url)
from mobilede_utils import request_mobile_de_page
from mobilede_parser import extract_car_data

url = '/fahrzeuge/details.html?id=372947832&cn=DE&fuels=ELECTRICITY&isSearchRequest=true&pageNumber=1&scopeId

In [95]:
from bs4 import BeautifulSoup
import unicodedata
import re

def _parse_number(string):
    """Parse e.g. 43.400 € to 43400 or 12.500 km to 12500"""
    return ("").join(re.findall(r"\d+", string))

def _text_or_none(element):
    return element.get_text() if element else None

def extract_car_data2(html):
    page_n = unicodedata.normalize("NFKD", html)
    soup = BeautifulSoup(page_n, 'html.parser')

    data = {}
    data["brand"], data["model"] = soup.find(id="ad-title").get_text().split(" ", 1)
    data["color"], data["color_type"], *_ =  s.get_text().split() + ["Nonmetallic"] if (s:=soup.find(id="color-v")) else [None, None]
    data["kilometers"] = _parse_number(soup.find(id="mileage-v").get_text())
    data["price"] = _parse_number(soup.find("span", {"data-testid" : "prime-price"}).get_text())
    data["registration"] = soup.find(id="firstRegistration-v").get_text()
    data["electric"] = soup.find(id="fuel-v").get_text() == "Elektro"
    data["power_kw"] = re.findall(r"\d+", soup.find(id="power-v").get_text())[0]
    data["capacity_kwh"] = _parse_number(s.get_text()) if (s:= soup.find(id="batteryCapacity-v")) else None
    data["automatic"] = s.get_text() == "Automatik" if (s:=soup.find(id="transmission-v")) else None
    data["undamaged"] = s.get_text() == "Unfallfrei" if  (s := soup.find(id="damageCondition-v")) else None

    data["el_cons_kwh100km"] = None
    if (s:= soup.find(id="envkv.wltp.powerConsumption-v")):
        data["el_cons_kwh100km"] = (".").join(re.findall(r"(\d+),(\d+)", s.get_text())[0])
    
    data["n_prev_owners"] = None
    if (s:= (soup.find("div", {"class" : "key-feature key-feature--numberOfPreviousOwners"})
                 .find("div", {"class" : "key-feature__value"}))):
        data["n_prev_owners"] = s.get_text()
    
    data["features"] = [item.get_text() for item in soup.find(id="features").find_all("div", {"class" : "bullet-list"})]
    if (s:= soup.find(id="parkAssists-v")):
        data["features"] = data["features"] + list(map(lambda x: f"Parking sensor: {x.strip()}", s.get_text().split(",")))

    #data["test"] = (soup.find(id="parkAssists-v"))
    #print({"Parking sensors" : (s.get_text() if (s:= soup.find(id="parkAssists-v")) else 0)})
    #data["test"] = 
    #data["features"].append({"Parking sensor" : (map(s.get_text().split(",")) if (s:= soup.find(id="parkAssists-v2")) else None)})
    #data["features"] = data["features"] + list(map(lambda x: f"Parking sensor: {x}", s.get_text().split(","))) if (s:= soup.find(id="parkAssists-v")) else None

    return data

extract_car_data2(html)

{'brand': 'Opel',
 'model': 'Astra',
 'color': 'Blau',
 'color_type': 'Metallic',
 'kilometers': '2500',
 'price': '47980',
 'registration': '06/2023',
 'electric': True,
 'power_kw': '115',
 'capacity_kwh': None,
 'automatic': True,
 'undamaged': True,
 'el_cons_kwh100km': '14.8',
 'n_prev_owners': '1',
 'features': ['ABS',
  'Abstandstempomat',
  'Abstandswarner',
  'Ambiente-Beleuchtung',
  'Android Auto',
  'Apple CarPlay',
  'Armlehne',
  'Beheizbare Frontscheibe',
  'Beheizbares Lenkrad',
  'Berganfahrassistent',
  'Blendfreies Fernlicht',
  'Bluetooth',
  'Bordcomputer',
  'Elektr. Fensterheber',
  'Elektr. Seitenspiegel',
  'Elektr. Wegfahrsperre',
  'ESP',
  'Freisprecheinrichtung',
  'Garantie',
  'Head-Up Display',
  'Induktionsladen für Smartphones',
  'Innenspiegel autom. abblendend',
  'Isofix',
  'Lederlenkrad',
  'LED-Scheinwerfer',
  'LED-Tagfahrlicht',
  'Leichtmetallfelgen',
  'Lichtsensor',
  'Lordosenstütze',
  'Müdigkeitswarner',
  'Multifunktionslenkrad',
  'N

In [101]:
def parse(html):
    page_n = unicodedata.normalize("NFKD", html)
    soup = BeautifulSoup(page_n, 'html.parser')

    data = {}
    data["brand"], data["model"] = soup.find(id="ad-title").get_text().split(" ", 1)
    data["color"], data["color_type"], *_ =  s.get_text().split() + ["Nonmetallic"] if (s:=soup.find(id="color-v")) else [None, None]
    data["kilometers"] = _parse_number(soup.find(id="mileage-v").get_text())
    data["price"] = _parse_number(soup.find("span", {"data-testid" : "prime-price"}).get_text())
    data["registration"] = soup.find(id="firstRegistration-v").get_text()
    data["electric"] = soup.find(id="fuel-v").get_text() == "Elektro"
    data["power_kw"] = re.findall(r"\d+", soup.find(id="power-v").get_text())[0]
    data["capacity_kwh"] = _parse_number(s.get_text()) if (s:= soup.find(id="batteryCapacity-v")) else None
    data["automatic"] = s.get_text() == "Automatik" if (s:=soup.find(id="transmission-v")) else None
    data["undamaged"] = s.get_text() == "Unfallfrei" if (s := soup.find(id="damageCondition-v")) else None
    s = soup.find(id="envkv.wltp.powerConsumption-v")
    data["el_cons_kwh100km"] = (".").join(re.findall(r"(\d+),(\d+)", s.get_text())[0]) if s else None
    s = (soup.find("div", {"class" : "key-feature key-feature--numberOfPreviousOwners"})
             .find("div", {"class" : "key-feature__value"}))
    data["n_prev_owners"] = s.get_text() if s else None
    data["features"] = [item.get_text() for item in soup.find(id="features").find_all("div", {"class" : "bullet-list"})]
    if (s:= soup.find(id="parkAssists-v")):
        data["features"] = data["features"] + list(map(lambda x: f"Parking sensor: {x.strip()}", s.get_text().split(",")))
        
    return data

parse(html)

{'brand': 'Opel',
 'model': 'Astra',
 'color': 'Blau',
 'color_type': 'Metallic',
 'kilometers': '2500',
 'price': '47980',
 'registration': '06/2023',
 'electric': True,
 'power_kw': '115',
 'capacity_kwh': None,
 'automatic': True,
 'undamaged': True,
 'el_cons_kwh100km': '14.8',
 'n_prev_owners': '1',
 'features': ['ABS',
  'Abstandstempomat',
  'Abstandswarner',
  'Ambiente-Beleuchtung',
  'Android Auto',
  'Apple CarPlay',
  'Armlehne',
  'Beheizbare Frontscheibe',
  'Beheizbares Lenkrad',
  'Berganfahrassistent',
  'Blendfreies Fernlicht',
  'Bluetooth',
  'Bordcomputer',
  'Elektr. Fensterheber',
  'Elektr. Seitenspiegel',
  'Elektr. Wegfahrsperre',
  'ESP',
  'Freisprecheinrichtung',
  'Garantie',
  'Head-Up Display',
  'Induktionsladen für Smartphones',
  'Innenspiegel autom. abblendend',
  'Isofix',
  'Lederlenkrad',
  'LED-Scheinwerfer',
  'LED-Tagfahrlicht',
  'Leichtmetallfelgen',
  'Lichtsensor',
  'Lordosenstütze',
  'Müdigkeitswarner',
  'Multifunktionslenkrad',
  'N

In [104]:
page_n = unicodedata.normalize("NFKD", html)
soup = BeautifulSoup(page_n, 'html.parser')

In [164]:

def get_capacity(soup):
    res = _parse_number(s.get_text()) if (s:= soup.find(id="batteryCapacity-v")) else []

    if (res == []):
        s = soup.find("div", {"class": "listing-title"}).find("div", {"class": "listing-subtitle"})
        res = re.findall("(\d{2,3})\s*kWh", s.get_text(), re.I) if s else []

    if (res == []):
        s = soup.find("div", {"class", "g-col-12 description"})
        res = re.findall("(\d{2,3})\s*kWh", s.get_text(), re.I) if s else []

    if len(res):
        return sorted(res, reverse=True)[0]

    return None


#if (s1 == []):
#    s1 = re.findall("\d{2,3}\s*kWh", s.get_text(), re.I)

#sorted(s1, reverse=True)[0]
#s1 = re.findall("\d{2,3}\s*kWh", s, re.I)
#if s1 == []:

s1

['40 kWh', '50kwh']

In [ ]:
if s not:


In [165]:
get_capacity(soup)

'40'

In [157]:
s = soup.find("div", {"class", "g-col-12 description"}).get_text() + " 40 kWh , 50kwh "
s1 = re.findall("\d{2,3}\s*kWh", s, re.I)
sorted(s1, reverse=True)[0]

'50kwh'

In [186]:
def _parse_number(string):
    """Parse e.g. 43.400 € to 43400 or 12.500 km to 12500"""
    return ("").join(re.findall(r"\d+", string))

def _parse_capacity(s):
    return re.findall("(\d{2,3})\s*kWh", s.get_text(), re.I) if s else []

def _get_capacity(soup):
    res = _parse_capacity(soup.find(id="batteryCapacity-v"))

    if res == []:
        res = _parse_capacity(soup.find("div", {"class": "listing-title"}).find("div", {"class": "listing-subtitle"}))
    if res == []:
        res = _parse_capacity(soup.find("div", {"class", "g-col-12 description"}))
    if len(res):
        return sorted(res, reverse=True)[0]

    return None

def extract_car_data(file):
    with open(file) as f:
        #html = unicodedata.normalize("NFKD", f)
        soup = BeautifulSoup(f, 'html.parser')
    #page_n = unicodedata.normalize("NFKD", html)
    

    data = {}
    data["brand"], data["model"] = soup.find(id="ad-title").get_text().split(" ", 1)
    data["color"], data["color_type"], *_ =  s.get_text().split() + ["Nonmetallic"] if (s:=soup.find(id="color-v")) else [None, None]
    data["kilometers"] = _parse_number(soup.find(id="mileage-v").get_text())
    data["price"] = _parse_number(soup.find("span", {"data-testid" : "prime-price"}).get_text())
    data["registration"] = s.get_text() if (s:=soup.find(id="firstRegistration-v")) else None
    data["electric"] = soup.find(id="fuel-v").get_text() == "Elektro"
    data["power_kw"] = re.findall(r"\d+", soup.find(id="power-v").get_text())[0]
    data["capacity_kwh"] = _get_capacity(soup)
    data["automatic"] = s.get_text() == "Automatik" if (s:=soup.find(id="transmission-v")) else None
    data["undamaged"] = s.get_text() == "Unfallfrei" if (s := soup.find(id="damageCondition-v")) else None
    s= soup.find(id="envkv.wltp.powerConsumption-v")
    data["el_cons_kwh100km"] = re.search(r"(\d+,\d+)", s.get_text()) if s else None
    #data["el_cons_kwh100km"] = (".").join(re.search(r"(\d+),(\d+)", s.get_text())) if s else None
    s = (soup.find("div", {"class" : "key-feature key-feature--numberOfPreviousOwners"})
             .find("div", {"class" : "key-feature__value"}))
    data["n_prev_owners"] = s.get_text() if s else None
    data["n_seats"] = s.get_text() if (s:= soup.find(id="numSeats-v")) else None
    data["features"] = [item.get_text() for item in soup.find(id="features").find_all("div", {"class" : "bullet-list"})]
    if (s:= soup.find(id="parkAssists-v")):
        data["features"] = data["features"] + list(map(lambda x: f"Parking sensor: {x.strip()}", s.get_text().split(",")))
        
    return data

In [188]:
extract_car_data("car3.html")

AttributeError: 'NoneType' object has no attribute 'get_text'

In [181]:
"".replace("c", "a")

''